In [6]:
import datetime
import os
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from notebook_shared import utils

from statsmodels.tsa.seasonal import seasonal_decompose
from pandas import Series

import seaborn as sns
sns.set()
matplotlib.style.use('seaborn-colorblind')

INPUT_FILE = "dataset"
FSIZE = "tiny"

df = pd.read_parquet(utils.get_dataset_path(INPUT_FILE, FSIZE))
mdf = pd.read_parquet(utils.get_dataset_path("mstatsdf", "full"))

In [9]:
# dirty_measurement df
dmdf = df[['driver_invocation', 'workload_invocation', 'provider', 'dirty_measurement', 'folder_uuid']]
dmdf

,driver_invocation,workload_invocation,provider,dirty_measurement,folder_uuid
0,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
1,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
2,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
3,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
4,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
...,...,...,...,...,...
499531,2021-09-30 23:30:03.438,2021-09-30 23:30:22.629,GCP,True,c0093932-341b-11ec-be3c-8bf2eba3159e
499532,2021-09-30 23:30:03.438,2021-09-30 23:30:22.629,GCP,True,c0093932-341b-11ec-be3c-8bf2eba3159e
499533,2021-09-30 23:30:03.438,2021-09-30 23:30:22.629,GCP,True,c0093932-341b-11ec-be3c-8bf2eba3159e
499534,2021-09-30 23:30:03.438,2021-09-30 23:30:22.629,GCP,True,c0093932-341b-11ec-be3c-8bf2eba3159e


In [24]:
dmdf[['provider', 'dirty_measurement', 'driver_invocation']].groupby(['provider','dirty_measurement']).count()

driver_invocation
provider dirty_measurement                   
AWS      False                           9954
         True                          163612
AZURE    False                           8252
         True                          143710
GCP      False                          10416
         True                          163592

In [26]:
mdf
# count all region occurences where value = 0 --> measurement missing
# count alle region occurecnes where value = 1 --> measurement ok
# count all regions occurences where value != 1 --> measurement 'dirty'
# count all region occurneces where value > 1 --> measured too often

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
count,3054.000000,3054.000000,3054.000000,3054.000000,3054.0,3054.000000,3054.000000,3054.0,3054.000000,3054.000000,...,3054.0,3054.000000,3054.0,3054.000000,3054.000000,3054.000000,3054.000000,3054.000000,3054.000000,3054.0
mean,0.993124,0.999673,0.997708,0.998690,1.0,0.995416,0.992141,1.0,0.997380,0.995088,...,1.0,0.997708,1.0,0.998035,0.998035,0.999673,0.998035,0.998363,0.994434,1.0
std,0.082651,0.018095,0.054246,0.036173,0.0,0.067562,0.111288,0.0,0.051122,0.069922,...,0.0,0.047829,0.0,0.044288,0.044288,0.018095,0.057198,0.040436,0.074413,0.0
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.000000,...,1.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
50%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
75%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
max,1.000000,1.000000,2.000000,1.000000,1.0,1.000000,2.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.0
